In [ ]:
import osmnx as ox
import networkx as nx
from operator import itemgetter

%matplotlib inline
G = ox.graph_from_place('Gentilly, France', network_type='drive')
G = nx.convert_node_labels_to_integers(G)
list_ = []
path = dict(nx.all_pairs_shortest_path(G))

def dfs(graph, node, visited):
    if node not in visited:
        visited.append(node)
        graph[node].reverse()
        for n in graph[node]:
            dfs(graph,n, visited)
    return visited

all_path = []
for elem in path.values():
    for var in elem.values():
        if len(var) > 3:
            all_path.append(var)

def rentability(checked_mat, need_to_happend):
    res = 0
    for elem in need_to_happend:
        if checked_mat[elem] != 1:
            res+=1
    return res/len(need_to_happend)

def update(checked_mat, res_path):
    for elem in res_path:
        if checked_mat[elem] != True:
            checked_mat[elem] = True
    return checked_mat
    
def find_critical_road(all_path):
    checked_mat = [False] * len(path.values())
    critical_floor = 0.9
    acceptable_len = 20
    critical_road = []
    length = len(all_path) - 1
    i = 0
    while False in checked_mat:
        if i > length:
            i = 0
            critical_floor -= 0.05
            acceptable_len -= 1
            print("The complete cover is at :", (checked_mat.count(True)*100/len(checked_mat)), "% of its completion")
        if len(all_path[i]) > acceptable_len:
            if rentability(checked_mat, all_path[i]) > critical_floor:
                critical_road.append(all_path[i])
                fig, ax = ox.plot_graph_route(G, all_path[i], route_linewidth=4, node_size=1, bgcolor='k')
                checked_mat = update(checked_mat, all_path[i])
        i+=1
    return critical_road

res_path = find_critical_road(all_path)